<a href="https://colab.research.google.com/github/prk14/Fashion_Mnist/blob/main/CNN_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 5.1 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
df = keras.datasets.fashion_mnist

In [4]:
(train_im,train_labels),(test_im, test_labels) = df.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
train_images = train_im/255.0
test_images = test_im/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', 
                                                          values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [12]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [17]:
tuner_search=RandomSearch(build_model,
                          objective='accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json


In [18]:
tuner_search.search(train_im, train_labels)

Trial 6 Complete [00h 00m 20s]
accuracy: 0.8183333277702332

Best accuracy So Far: 0.8400166630744934
Total elapsed time: 00h 01m 44s
INFO:tensorflow:Oracle triggered exit


In [32]:
model = tuner_search.get_best_models(num_models=1)
model=model[0]

In [33]:
model.fit(train_im,train_labels,epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.2890 - accuracy: 0.8943 - val_loss: 0.2835 - val_accuracy: 0.8935
Epoch 5/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2231 - accuracy: 0.9158 - val_loss: 0.3158 - val_accuracy: 0.8878
Epoch 6/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.1788 - accuracy: 0.9325 - val_loss: 0.3064 - val_accuracy: 0.8923
Epoch 7/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.1443 - accuracy: 0.9469 - val_loss: 0.3717 - val_accuracy: 0.8955
Epoch 8/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.1137 - accuracy: 0.9586 - val_loss: 0.4331 - val_accuracy: 0.8847
Epoch 9/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0960 - accuracy: 0.9651 - val_loss: 0.4533 - val_accuracy: 0.8932
Epoch 10/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0760 - accuracy: 0.9736 - val_loss: 0.5496 